## Скачаем текст

In [1]:
import urllib.request, re

In [2]:
with urllib.request.urlopen('https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/zhivago.txt') as inp:
    text = inp.read().decode('utf-8')

## Задание 1.

In [3]:
def clear(text):
    text = re.sub(r'<(binary|program-used|src-url|lang|id|version|nickname|date)[^>]*>[^<]*</\1>', ' ', text)
    text = re.sub(r'<[^>]+>', ' ', text)
    text = text.replace('\xa0', ' ')
    text = re.sub('  +', ' ', text)
    return text.strip()

In [4]:
text = clear(text)

In [5]:
text[:2000]

'Борис Леонидович Пастернак \n Доктор Живаго \n «Доктор Живаго» - итоговое произведение Бориса Пастернака, книга всей его жизни. Этот роман принес его автору мировую известность и Нобелевскую премию, присуждение которой обернулось для поэта оголтелой политической травлей, обвинениями в «измене Родине» и в результате стоило ему жизни. \n «Доктор Живаго» - роман, сама ткань которого убедительнее свидетельствует о чуде, чем все размышления доктора и обобщения автора. Человек, который так пишет, бесконечно много пережил и передумал, и главные его чувства на свете - восхищенное умиление и слезное сострадание; конечно, есть в его мире место и презрению, и холодному отстранению - но не в них суть. Роман Пастернака - оплакивание прежних заблуждений и их жертв; те, кто не разделяет молитвенного восторга перед миром, достойны прежде всего жалости. Перечитывать «Доктора Живаго» стоит именно тогда, когда кажется, что жить не стоит. Тогда десять строк из этого романа могут сделать то же, что делает

## Задание 2.

In [6]:
from rusenttokenize import ru_sent_tokenize
from razdel import tokenize

In [7]:
text = text.lower()

In [8]:
def rm_punct(text):
    text = re.sub('\W',' ',text)
    text = re.sub('\s+',' ',text)
    return text.strip()

def my_tokenize(text):
    return [token.text for token in tokenize(text)]

In [9]:
rm_punct("Дорогие, уважаемые друзья-товарищи, давайте жить дружно!")

'Дорогие уважаемые друзья товарищи давайте жить дружно'

In [10]:
tokenized_sentences = [my_tokenize(rm_punct(sent)) for sent in ru_sent_tokenize(text)]

In [11]:
print(tokenized_sentences[:3])

[['борис', 'леонидович', 'пастернак', 'доктор', 'живаго', 'доктор', 'живаго', 'итоговое', 'произведение', 'бориса', 'пастернака', 'книга', 'всей', 'его', 'жизни'], ['этот', 'роман', 'принес', 'его', 'автору', 'мировую', 'известность', 'и', 'нобелевскую', 'премию', 'присуждение', 'которой', 'обернулось', 'для', 'поэта', 'оголтелой', 'политической', 'травлей', 'обвинениями', 'в', 'измене', 'родине', 'и', 'в', 'результате', 'стоило', 'ему', 'жизни'], ['доктор', 'живаго', 'роман', 'сама', 'ткань', 'которого', 'убедительнее', 'свидетельствует', 'о', 'чуде', 'чем', 'все', 'размышления', 'доктора', 'и', 'обобщения', 'автора']]


Найдём повторяющиеся предложения:

In [12]:
def find_rep_sents(sent_list):
    sent_list = [tuple(sent) for sent in sent_list]
    
    rep_sents = dict()
    
    for sent in set(sent_list):
        sent_count = sent_list.count(sent)
        if sent_count > 1:
            rep_sents[sent] = sent_count
    return rep_sents

In [13]:
rep_sents = find_rep_sents(tokenized_sentences)

In [14]:
rep_sents

{('странно',): 2,
 ('свеча', 'горела', 'на', 'столе', 'свеча', 'горела'): 3,
 ('да',): 2,
 ('толпа', 'росла'): 2,
 ('прорыв',): 2,
 ('он', 'открыл', 'глаза'): 2,
 ('парило',): 2}

Какой самый частотный токен в тексте длинее 6 символов?

In [15]:
from collections import Counter

In [16]:
freq_dict = Counter([tok for sent in tokenized_sentences for tok in sent if len(tok) > 6])

In [17]:
freq_dict.most_common(1)

[('андреевич', 289)]

## Задание 3

In [18]:
from nltk.stem.snowball import SnowballStemmer

In [19]:
stemmer = SnowballStemmer('russian')

In [20]:
stemmed_sentences = [[(word, stemmer.stem(word)) for word in sent] for sent in tokenized_sentences]

Рассмотрим какие варианты нашлись у каких стем:

In [21]:
tokens_n_stems = [word for sent in stemmed_sentences for word in sent]

In [22]:
tokens_n_stems[:20]

[('борис', 'борис'),
 ('леонидович', 'леонидович'),
 ('пастернак', 'пастернак'),
 ('доктор', 'доктор'),
 ('живаго', 'живаг'),
 ('доктор', 'доктор'),
 ('живаго', 'живаг'),
 ('итоговое', 'итогов'),
 ('произведение', 'произведен'),
 ('бориса', 'борис'),
 ('пастернака', 'пастернак'),
 ('книга', 'книг'),
 ('всей', 'все'),
 ('его', 'ег'),
 ('жизни', 'жизн'),
 ('этот', 'этот'),
 ('роман', 'рома'),
 ('принес', 'принес'),
 ('его', 'ег'),
 ('автору', 'автор')]

Найдём для каждой основы её частоту, её варианты и частоты вариантов:

In [23]:
stem_freqs = []

In [24]:
from collections import defaultdict

In [25]:
stems = set([i[1] for i in tokens_n_stems])

In [26]:
from tqdm import tqdm_notebook

In [27]:
for stem in tqdm_notebook(stems,total=len(stems)):
    stem_info = {'stem': stem}
    stem_variants = [entry[0] for entry in tokens_n_stems if entry[1] == stem]
    stem_info['frequency'] = len(stem_variants)
    variant_freqs = dict(Counter(stem_variants))
    stem_info['variant_freqs'] = variant_freqs
    stem_freqs.append(stem_info)

C:\Users\k1l77\python_envs\diploma_env\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [28]:
stem_freqs = sorted(stem_freqs, key=lambda x: x['frequency'], reverse=True)

Чтобы не засорять ноутбук, сохраним отсортированные по частоте основы с вариантами в отдельный файл. Наблюдение ниже основано на данных этого файла

In [29]:
import json

In [46]:
with open('stem_freqs.json', 'w', encoding='utf-8') as outp:
    json.dump(stem_freqs, outp, ensure_ascii=False, indent=4)

Частые ошибки:

    друг и другой; надо, надеюсь и Надя; меня и менее; да и дай; часть и частый; сторона и сторониться; вас и Вася; пол, полу-, полый, полено и поле

Найдём слова более 4 символов, которые не изменились после стемминга:

In [32]:
not_changed = [entry for entry in stem_freqs if entry['stem'] in entry['variant_freqs'] and len(entry['stem'])>4]

In [33]:
not_changed = sorted(not_changed, key=lambda x: x['frequency'], reverse=True)

Сохраним отобранные основы в файл:

In [47]:
with open('not_changed.json', 'w', encoding='utf-8') as outp:
    json.dump(not_changed, outp, ensure_ascii=False, indent=4)

На основе наблюдений над файлом получим следующий вывод:

Слова, не изменившиеся после стемминга: <i>может</i>, <i>сидел</i>, <i>ветер</i>, <i>тянул</i>, <i>ненавидел</i>, <i>построек</i>, <i>чисел</i>, <i>винтовок</i>, <i>непорядок</i>

## Задание 5

In [35]:
import nltk

In [36]:
from nltk.corpus import stopwords

In [37]:
print(sorted(stopwords.words('russian')))

['а', 'без', 'более', 'больше', 'будет', 'будто', 'бы', 'был', 'была', 'были', 'было', 'быть', 'в', 'вам', 'вас', 'вдруг', 'ведь', 'во', 'вот', 'впрочем', 'все', 'всегда', 'всего', 'всех', 'всю', 'вы', 'где', 'да', 'даже', 'два', 'для', 'до', 'другой', 'его', 'ее', 'ей', 'ему', 'если', 'есть', 'еще', 'ж', 'же', 'за', 'зачем', 'здесь', 'и', 'из', 'или', 'им', 'иногда', 'их', 'к', 'как', 'какая', 'какой', 'когда', 'конечно', 'кто', 'куда', 'ли', 'лучше', 'между', 'меня', 'мне', 'много', 'может', 'можно', 'мой', 'моя', 'мы', 'на', 'над', 'надо', 'наконец', 'нас', 'не', 'него', 'нее', 'ней', 'нельзя', 'нет', 'ни', 'нибудь', 'никогда', 'ним', 'них', 'ничего', 'но', 'ну', 'о', 'об', 'один', 'он', 'она', 'они', 'опять', 'от', 'перед', 'по', 'под', 'после', 'потом', 'потому', 'почти', 'при', 'про', 'раз', 'разве', 'с', 'сам', 'свою', 'себе', 'себя', 'сейчас', 'со', 'совсем', 'так', 'такой', 'там', 'тебя', 'тем', 'теперь', 'то', 'тогда', 'того', 'тоже', 'только', 'том', 'тот', 'три', 'тут', 'ты

Также следует добавить слова:
    
     ними - Местоимение 3л. Мн. ч. Тв. п. Остальные личные местоимения в списке есть
     снова -  В списке есть "опять"
     ничем, ничему - формы от "ничего"
     тому - Д.п. от "тот", отсутствующая форма в списке
     тобой - Тв. п. от "ты", отсутствующая форма в списке

## Задание 6

In [38]:
with open('input_text.txt', 'w', encoding='utf-8') as outp:
    outp.write(text)

In [39]:
!mystem -d --format json input_text.txt text_parsed.json

In [40]:
with open('text_parsed.json', 'r', encoding='utf-8') as inp:
    json_out = inp.read()

Исправим mystem'овский json так, чтобы он был валидным:

In [41]:
json_out = '[' + re.sub('\n(?!")',',\n',json_out)[:-2] + ']'

In [44]:
mystem_lemmatized = json.loads(json_out)

In [56]:
mystem_lemmatized[:2]

[[{'analysis': [{'lex': 'борис'}], 'text': 'борис'},
  {'analysis': [{'lex': 'леонидович'}], 'text': 'леонидович'},
  {'analysis': [{'lex': 'пастернак'}], 'text': 'пастернак'}],
 [{'analysis': [{'lex': 'доктор'}], 'text': 'доктор'},
  {'analysis': [{'lex': 'живаго'}], 'text': 'живаго'}]]

Сделаем то же самое при помощи pymorphy:

In [49]:
from pymorphy2 import MorphAnalyzer

In [50]:
morph = MorphAnalyzer()

In [51]:
pymorphy_lemmatized = [[(token, morph.parse(token)[0].normal_form) for token in sent] for sent in tokenized_sentences]

In [57]:
pymorphy_lemmatized[:2]

[[('борис', 'борис'),
  ('леонидович', 'леонидович'),
  ('пастернак', 'пастернак'),
  ('доктор', 'доктор'),
  ('живаго', 'живаго'),
  ('доктор', 'доктор'),
  ('живаго', 'живаго'),
  ('итоговое', 'итоговый'),
  ('произведение', 'произведение'),
  ('бориса', 'борис'),
  ('пастернака', 'пастернак'),
  ('книга', 'книга'),
  ('всей', 'весь'),
  ('его', 'он'),
  ('жизни', 'жизнь')],
 [('этот', 'этот'),
  ('роман', 'роман'),
  ('принес', 'принести'),
  ('его', 'он'),
  ('автору', 'автор'),
  ('мировую', 'мировой'),
  ('известность', 'известность'),
  ('и', 'и'),
  ('нобелевскую', 'нобелевский'),
  ('премию', 'премия'),
  ('присуждение', 'присуждение'),
  ('которой', 'который'),
  ('обернулось', 'обернуться'),
  ('для', 'для'),
  ('поэта', 'поэт'),
  ('оголтелой', 'оголтелый'),
  ('политической', 'политический'),
  ('травлей', 'травля'),
  ('обвинениями', 'обвинение'),
  ('в', 'в'),
  ('измене', 'измена'),
  ('родине', 'родина'),
  ('и', 'и'),
  ('в', 'в'),
  ('результате', 'результат'),
  ('с

Произведём сравнение - посмотрим какие токены MyStem и PyMorphy лемматизировали по-разному:

In [68]:
def get_mystem_lemma(item):
    if item['analysis']:
        return item['analysis'][0]['lex']
    else:
        return None

mystem_flat = [(item['text'], get_mystem_lemma(item)) for sent in mystem_lemmatized for item in sent]
pymorphy_flat = [item for sent in pymorphy_lemmatized for item in sent]

In [69]:
print('Token ID', 'Mystem(Токен, Лемма)', 'PyMorphy(токен, Лемма)')

for idx, (item_mystem, item_pymorphy) in enumerate(zip(mystem_flat, pymorphy_flat)):
    if item_mystem[1] != item_pymorphy[1] and item_mystem[0] == item_pymorphy[0]:
        print(idx, item_mystem, item_pymorphy)

Token ID Mystem(Токен, Лемма) PyMorphy(токен, Лемма)
13 ('его', 'его') ('его', 'он')
17 ('принес', 'приносить') ('принес', 'принести')
27 ('обернулось', 'обертываться') ('обернулось', 'обернуться')
49 ('убедительнее', 'убедительно') ('убедительнее', 'убедительный')
54 ('все', 'весь') ('все', 'всё')
66 ('пережил', 'переживать') ('пережил', 'пережить')
68 ('передумал', 'передумывать') ('передумал', 'передумать')
71 ('его', 'его') ('его', 'он')
75 ('восхищенное', 'восхищенный') ('восхищенное', 'восхитить')
78 ('слезное', 'слезный') ('слезное', 'слёзный')
81 ('есть', 'быть') ('есть', 'есть')
83 ('его', 'его') ('его', 'он')
95 ('суть', 'суть') ('суть', 'быть')
102 ('их', 'их') ('их', 'они')
114 ('всего', 'все') ('всего', 'весь')
119 ('стоит', 'стоять') ('стоит', 'стоить')
132 ('этого', 'этот') ('этого', 'это')
150 ('беспричинно', 'беспричинный') ('беспричинно', 'беспричинно')
4607 ('его', 'его') ('его', 'он')
6523 ('его', 'его') ('его', 'он')
14310 ('остановился', 'останавливаться') ('остан

Результат сравнения:

 - MyStem - возводит глаголы к форме несовершенного вида, в то время как PyMorphy сохраняет вид
 - MyStem в отличие от PyMorphy не возводит к форме именительного падежа личные местоимения, но при этом возводит местоимение <i>все</i> к форме <i>весь</i>

Интересные случаи:

 ID 49 убедительнее - убедительно (MyStem) vs. убедительный (PyMorphy)
 
 ID 75 восхищенное - восхищенный (MyStem) vs. восхитить (PyMorphy)
 
 ID 81 есть - быть (MyStem) vs. есть (PyMorphy)
 
 ID 95 суть - суть (MyStem) vs. быть (PyMorphy)
 
 ID 150 беспричинно - беспричинный (MyStem) vs. беспричинно (PyMorphy)

In [71]:
for idx, (item_mystem, item_pymorphy) in enumerate(zip(mystem_flat[44:58], pymorphy_flat[44:58])):
    print(idx, item_mystem, item_pymorphy)

0 ('живаго', 'живаго') ('живаго', 'живаго')
1 ('роман', 'роман') ('роман', 'роман')
2 ('сама', 'сам') ('сама', 'сам')
3 ('ткань', 'ткань') ('ткань', 'ткань')
4 ('которого', 'который') ('которого', 'который')
5 ('убедительнее', 'убедительно') ('убедительнее', 'убедительный')
6 ('свидетельствует', 'свидетельствовать') ('свидетельствует', 'свидетельствовать')
7 ('о', 'о') ('о', 'о')
8 ('чуде', 'чудо') ('чуде', 'чудо')
9 ('чем', 'чем') ('чем', 'чем')
10 ('все', 'весь') ('все', 'всё')
11 ('размышления', 'размышление') ('размышления', 'размышление')
12 ('доктора', 'доктор') ('доктора', 'доктор')
13 ('и', 'и') ('и', 'и')


In [72]:
for idx, (item_mystem, item_pymorphy) in enumerate(zip(mystem_flat[70:80], pymorphy_flat[70:80])):
    print(idx, item_mystem, item_pymorphy)

0 ('главные', 'главный') ('главные', 'главный')
1 ('его', 'его') ('его', 'он')
2 ('чувства', 'чувство') ('чувства', 'чувство')
3 ('на', 'на') ('на', 'на')
4 ('свете', 'свет') ('свете', 'свет')
5 ('восхищенное', 'восхищенный') ('восхищенное', 'восхитить')
6 ('умиление', 'умиление') ('умиление', 'умиление')
7 ('и', 'и') ('и', 'и')
8 ('слезное', 'слезный') ('слезное', 'слёзный')
9 ('сострадание', 'сострадание') ('сострадание', 'сострадание')


In [73]:
for idx, (item_mystem, item_pymorphy) in enumerate(zip(mystem_flat[80:90], pymorphy_flat[80:90])):
    print(idx, item_mystem, item_pymorphy)

0 ('конечно', 'конечно') ('конечно', 'конечно')
1 ('есть', 'быть') ('есть', 'есть')
2 ('в', 'в') ('в', 'в')
3 ('его', 'его') ('его', 'он')
4 ('мире', 'мир') ('мире', 'мир')
5 ('место', 'место') ('место', 'место')
6 ('и', 'и') ('и', 'и')
7 ('презрению', 'презрение') ('презрению', 'презрение')
8 ('и', 'и') ('и', 'и')
9 ('холодному', 'холодный') ('холодному', 'холодный')


In [74]:
for idx, (item_mystem, item_pymorphy) in enumerate(zip(mystem_flat[90:100], pymorphy_flat[90:100])):
    print(idx, item_mystem, item_pymorphy)

0 ('отстранению', 'отстранение') ('отстранению', 'отстранение')
1 ('но', 'но') ('но', 'но')
2 ('не', 'не') ('не', 'не')
3 ('в', 'в') ('в', 'в')
4 ('них', 'они') ('них', 'они')
5 ('суть', 'суть') ('суть', 'быть')
6 ('роман', 'роман') ('роман', 'роман')
7 ('пастернака', 'пастернак') ('пастернака', 'пастернак')
8 ('оплакивание', 'оплакивание') ('оплакивание', 'оплакивание')
9 ('прежних', 'прежний') ('прежних', 'прежний')


In [75]:
for idx, (item_mystem, item_pymorphy) in enumerate(zip(mystem_flat[145:155], pymorphy_flat[145:155])):
    print(idx, item_mystem, item_pymorphy)

0 ('доктора', 'доктор') ('доктора', 'доктор')
1 ('жизнь', 'жизнь') ('жизнь', 'жизнь')
2 ('вернулась', 'вернуться') ('вернулась', 'вернуться')
3 ('так', 'так') ('так', 'так')
4 ('же', 'же') ('же', 'же')
5 ('беспричинно', 'беспричинный') ('беспричинно', 'беспричинно')
6 ('как', 'как') ('как', 'как')
7 ('когда-то', 'когда-то') ('когда', 'когда')
8 ('странно', 'странно') ('то', 'то')
9 ('прервалась', 'прерываться') ('странно', 'странно')


<b>Вывод</b>: В целои, MyStem и PyMorphy одинаково хорошо справляются с лемматизацией текста. Недостатками MyStem перед PyMorphy являются возведение всех глаголов к форме несовершенного вида и отсутствие приведения к нормальной форме. Преимуществом MyStem перед PyMorphy является лемматизация с учётом контекста (см. пример с <i>суть</i>/<i>быть</i>.